In [ ]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [46]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

# Muat data
data_dict = pickle.load(open('../data.pickle', 'rb'))

# Periksa bentuk dari setiap elemen dalam data_dict['data']
# Buat daftar untuk menyimpan data yang valid
valid_data = []

for sample in data_dict['data']:
    if len(sample) == 42:  # Pastikan panjang fitur adalah 42
        valid_data.append(sample)

# Konversi ke array NumPy
data = np.array(valid_data)  # Harus berukuran (n_samples, 42)
labels = np.array(data_dict['labels'][:len(valid_data)])  # Sesuaikan dengan jumlah data valid

# Membuat labels_dict untuk pemetaan huruf
labels_dict = {i: label for i, label in enumerate(sorted(set(labels)))}

# Split data menjadi training dan testing
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

# Inisialisasi dan latih model
model = RandomForestClassifier()

# Data sudah memiliki bentuk yang sesuai
model.fit(x_train, y_train)

# Prediksi dan hitung skor akurasi
y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)
print('{}% of samples were classified correctly !'.format(score * 100))

# Cek apakah file model.p sudah ada
model_filename = 'model.p'
if os.path.exists(model_filename):
    # Jika ada, rename menjadi model1.p, model2.p, dst.
    i = 1
    new_filename = f'{os.path.splitext(model_filename)[0]}{i}{os.path.splitext(model_filename)[1]}'
    while os.path.exists(new_filename):
        i += 1
        new_filename = f'{os.path.splitext(model_filename)[0]}{i}{os.path.splitext(model_filename)[1]}'
    os.rename(model_filename, new_filename)

# Tentukan folder untuk menyimpan model
model_folder = '../model/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

# Tentukan nama file model dengan increment
i = 1
model_filename = os.path.join(model_folder, 'rf_model.p')
while os.path.exists(model_filename):
    model_filename = os.path.join(model_folder, f'rf_model_{i}.p')
    i += 1

# Simpan model ke file
with open(model_filename, 'wb') as f:
    pickle.dump({'model': model}, f)

97.29155602761551% of samples were classified correctly !


In [ ]:
# # NEURAL NETWORK

# from sklearn.neural_network import MLPClassifier

# # Inisialisasi model dengan parameter yang sesuai
# mlp_model = MLPClassifier(max_iter=1, warm_start=True)

# # Latih model selama 100 epoch
# for epoch in range(100):
#     mlp_model.fit(x_train, y_train)
#     y_predict = mlp_model.predict(x_test)
#     score = accuracy_score(y_test, y_predict)
#     print(f'Epoch {epoch + 1}: {score * 100:.2f}% of samples were classified correctly!')

# # Simpan model ke file
# model_filename = os.path.join(model_folder, 'mlp_model.p')
# with open(model_filename, 'wb') as f:
#     pickle.dump({'model': mlp_model}, f)

In [47]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_predict, target_names=[labels_dict[i] for i in sorted(labels_dict.keys())])
print(report)

              precision    recall  f1-score   support

           A       0.98      0.99      0.98        80
           B       0.97      0.97      0.97        80
           C       0.96      0.99      0.97        79
           D       1.00      0.99      0.99        77
           E       1.00      0.99      0.99        80
           F       0.99      0.95      0.97        78
           G       0.96      0.96      0.96        73
           H       0.99      0.97      0.98        80
           I       0.96      1.00      0.98        79
           J       0.98      0.96      0.97        45
           K       0.93      0.99      0.96        67
           L       0.99      0.99      0.99        80
           M       0.98      0.98      0.98        64
           N       0.98      0.95      0.97        59
           O       0.94      0.97      0.95        76
           P       0.96      0.99      0.97        74
           Q       0.99      0.95      0.97        75
           R       0.94    